In [6]:
%idle_timeout 2880
%glue_version 4.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

You are already connected to a glueetl session 0b8f5907-1d0d-47ae-a141-9a93a8338e15.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.


You are already connected to a glueetl session 0b8f5907-1d0d-47ae-a141-9a93a8338e15.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Setting Glue version to: 4.0


You are already connected to a glueetl session 0b8f5907-1d0d-47ae-a141-9a93a8338e15.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous worker type: None
Setting new worker type to: G.1X


You are already connected to a glueetl session 0b8f5907-1d0d-47ae-a141-9a93a8338e15.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous number of workers: None
Setting new number of workers to: 5



In [31]:
from pyspark.sql.functions import explode, col, to_date
from datetime import datetime
from awsglue.dynamicframe import DynamicFrame

In [21]:
s3_path = "s3://spotify-etl-walker/raw_data/to_processed/"
source_dyf = glueContext.create_dynamic_frame_from_options(
    connection_type="s3",
    connection_options={"paths":[s3_path]},
    format="json"
)        

In [22]:
spotify_df = source_dyf.toDF()

In [23]:
def process_albums(df):
    df = df.withColumn("items", explode("items")).select(
        col("items.track.album.id").alias("album_id"),
        col("items.track.album.name").alias("album_name"),
        col("items.track.album.release_date").alias("release_date"),
        col("items.track.album.total_tracks").alias("total_tracks"),
        col("items.track.album.external_urls.spotify").alias("url")
    ).drop_duplicates(["album_id"])
    return df


def process_artists(df):
    # First, explode the items to get individual tracks
    df_items_exploded = df.select(explode(col("items")).alias("item"))
    
    # Then, explode the artists array within each item to create a row for each artist
    df_artists_exploded = df_items_exploded.select(explode(col("item.track.artists")).alias("artist"))
    
    # Now, select the artist attributes, ensuring each artist is in its own row
    df_artists = df_artists_exploded.select(
        col("artist.id").alias("artist_id"),
        col("artist.name").alias("artist_name"),
        col("artist.external_urls.spotify").alias("external_url")
    ).drop_duplicates(["artist_id"])
    
    return df_artists


def process_songs(df):
    # Explode the items array to create a row for each song
    df_exploded = df.select(explode(col("items")).alias("item"))
    
    # Extract song information from the exploded DataFrame
    df_songs = df_exploded.select(
        col("item.track.id").alias("song_id"),
        col("item.track.name").alias("song_name"),
        col("item.track.duration_ms").alias("duration_ms"),
        col("item.track.external_urls.spotify").alias("url"),
        col("item.track.popularity").alias("popularity"),
        col("item.added_at").alias("song_added"),
        col("item.track.album.id").alias("album_id"),
        col("item.track.artists")[0]["id"].alias("artist_id")
    ).drop_duplicates(["song_id"])
    
    # Convert string dates in 'song_added' to actual date types
    df_songs = df_songs.withColumn("song_added", to_date(col("song_added")))
    
    return df_songs


In [24]:
album_df = process_albums(spotify_df)
album_df.show(5)

+--------------------+--------------------+------------+------------+--------------------+
|            album_id|          album_name|release_date|total_tracks|                 url|
+--------------------+--------------------+------------+------------+--------------------+
|0DLvFVIfwt3OHdK9k...|Where I've Been, ...|  2024-05-31|          12|https://open.spot...|
|0QZVRL2HC24voXfxW...|             Madonna|  2024-03-08|           1|https://open.spot...|
|0lgs2Sa82lyX89nBU...|      FERXXOCALIPSIS|  2023-12-01|          10|https://open.spot...|
|168CdR21lfn0TTyw1...|Fireworks & Rolle...|  2024-04-05|          15|https://open.spot...|
|18NOKLkZETa4sWwLM...|              UTOPIA|  2023-07-28|          19|https://open.spot...|
+--------------------+--------------------+------------+------------+--------------------+
only showing top 5 rows


In [25]:
artist_df = process_artists(spotify_df)
artist_df.show(5)

+--------------------+-------------+--------------------+
|           artist_id|  artist_name|        external_url|
+--------------------+-------------+--------------------+
|06HL4z0CvFAxyc27G...| Taylor Swift|https://open.spot...|
|0PCCGZ0wGLizHt2KZ...|      Artemas|https://open.spot...|
|0Y5tJX1MQlPlqiwlO...| Travis Scott|https://open.spot...|
|0du5cEVh5yTK9QJze...|   Bruno Mars|https://open.spot...|
|0elWFr7TW8piilVRY...|Natanael Cano|https://open.spot...|
+--------------------+-------------+--------------------+
only showing top 5 rows


In [26]:
song_df = process_songs(spotify_df)
song_df.show(5)

+--------------------+----------------+-----------+--------------------+----------+----------+--------------------+--------------------+
|             song_id|       song_name|duration_ms|                 url|popularity|song_added|            album_id|           artist_id|
+--------------------+----------------+-----------+--------------------+----------+----------+--------------------+--------------------+
|0WbMK4wrZ1wFSty9F...|Good Luck, Babe!|     218423|https://open.spot...|        91|2024-06-03|1WAjjRMfZjEXtB0lQ...|7GlBOeep6PqTfFi59...|
|0mflMxspEfB0VbI1k...|    Stick Season|     182346|https://open.spot...|        92|2024-06-03|50ZenUP4O2Q5eCy2N...|2RQXRUsr4IW1f3mKy...|
|17phhZDn6oGtzMe56...|    Lose Control|     210688|https://open.spot...|        92|2024-06-03|7nacKlk586eLRBSIs...|33qOK5uJ8AR2xuQQA...|
|1BxfuPKGuaTgP7aM0...|    Cruel Summer|     178426|https://open.spot...|        94|2024-06-03|1NAmidJlEaVgA3Mpc...|06HL4z0CvFAxyc27G...|
|1C84d9abZVKWHT2YY...|Si No Quieres No|  

In [42]:
def write_to_s3(df, path_suffix, format_type="csv"):
    # Convert back to DynamicFrame
    dynamic_frame = DynamicFrame.fromDF(df, glueContext, "dynamic_frame")
    
    glueContext.write_dynamic_frame.from_options(
        frame = dynamic_frame,
        connection_type = "s3",
        connection_options = {"path": f"s3://spotify-etl-walker/transformed_data/{path_suffix}/"},
        format = format_type
    )

In [43]:
write_to_s3(album_df, "album_data/album_transformed_{}".format(datetime.now().strftime("%Y-%m-%d")), "csv")

In [44]:
write_to_s3(artist_df, "artist_data/artist_transformed_{}".format(datetime.now().strftime("%Y-%m-%d")), "csv")

In [45]:
write_to_s3(song_df, "songs_data/songs_transformed_{}".format(datetime.now().strftime("%Y-%m-%d")), "csv")